In [1]:
import datetime
import json
import pandas as pd
import numpy as np
import spacy
import re
import math
from unidecode import unidecode
from collections import Counter
from pymongo import MongoClient
nlp = spacy.load("es_core_news_lg")

In [4]:
client = MongoClient('mongodb://localhost:27017')
database = client['Thesis']
collectionDB = database.get_collection("text")
collectionDep_2 = database.get_collection("text_dep2")

In [19]:
stop_words = pd.read_excel('../input/stop_words_lemma.xlsx')['stop_lemma'].to_list()

In [22]:
def preproccess_0(texto):
    texto = texto.replace('\n', ' ')
    texto = texto.replace('\t', ' ')
    texto = re.sub(' +', ' ', texto)
    minusculas = texto.lower()
    sin_tildes = unidecode(minusculas)

    # Step 1: Eliminar caracteres especiales.
    texto_sin_caracteres_especiales = re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ\s]", " ", sin_tildes)
    # Step 2: Eliminar palabras con más de 20 caracteres o con menos de 2.
    palabras = texto_sin_caracteres_especiales.split()
    palabras_filtradas = []
    for palabra in palabras:
        if len(palabra) > 1 and len(palabra) <= 20:
            palabras_filtradas.append(palabra)
    texto_filtrado = " ".join(palabras_filtradas) 
    
    if len(texto_filtrado) > 1000000:
        sin_tildes = " "
        for i in range(math.ceil(len(text)/1000000)):
            a = math.ceil(len(text)/math.ceil(len(text)/1000000))
            text_a = texto_filtrado[i*a:(i+1)*a]
            doc = nlp(text_a)
            palabras_lematizadas = [token.lemma_ for token in doc]
            text_lemma = " ".join(palabras_lematizadas)
            text_unicode = unidecode(text_lemma)
            sin_tildes = sin_tildes + " " +  text_unicode
    else:
        # Step 3: Lematizar las palabras.
        doc = nlp(texto_filtrado)
        palabras_lematizadas = [token.lemma_ for token in doc]
        text_lemma = " ".join(palabras_lematizadas)
        sin_tildes = unidecode(text_lemma)     
    # Step 3: Eliminar stopwords.
    palabras2 = sin_tildes.split()
    no_stop = [word for word in palabras2 if word not in stop_words]
    texto_sin_stopwords = " ".join(no_stop)  
    return texto_sin_stopwords

In [5]:
print("Start  -------------------------------")
print(datetime.datetime.now().time())
count = 0
for document in collectionDB.find():
    text = document["text"]
    number = document["file"]
    texto = preproccess_0(text)
    res = dict()
    res['version'] = "0.0.1"
    res['file'] = number
    res['text'] = texto
    result_object = collectionDep_2.insert_one(res)
    
print("End -------------------------------")
print(datetime.datetime.now().time())

Start  -------------------------------
12:30:11.254212


NameError: name 'preproccess_0' is not defined

In [ ]:
data.columns